In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("employee_data.csv")

# 1. Classify employees as "Underperforming", "Average", or "High Performer"
def performance_classification(projects):
    if projects <= 2:
        return "Underperforming"
    elif 3 <= projects <= 4:
        return "Average"
    else:
        return "High Performer"

df['Performance'] = df['Projects_Completed'].apply(performance_classification)

# 2. Group by Department to analyze average working hours and projects completed
department_analysis = df.groupby('Department').agg(
    Avg_Monthly_Hours_Worked=('Monthly_Hours_Worked', 'mean'),
    Avg_Projects_Completed=('Projects_Completed', 'mean')
).reset_index()

# 3. Aggregate salary data (total, average, and max) by department
salary_aggregation = df.groupby('Department').agg(
    Total_Salary=('Salary', 'sum'),
    Average_Salary=('Salary', 'mean'),
    Max_Salary=('Salary', 'max')
).reset_index()

# 4. Sort employees by Projects_Completed in descending order
sorted_employees = df.sort_values(by='Projects_Completed', ascending=False)

# 5. Create pivot table to compare average salary by department and performance classification
salary_pivot = pd.pivot_table(
    df,
    values='Salary',
    index='Department',
    columns='Performance',
    aggfunc='mean'
).fillna(0)

# Optional: Convert Joining_Date to datetime for further analysis
df['Joining_Date'] = pd.to_datetime(df['Joining_Date'])

# ---- Display outputs ----
print("\n🎯 Employee Performance Classification:")
print(df[['Employee_ID', 'Projects_Completed', 'Performance']])

print("\n📊 Department-Wise Performance Analysis:")
print(department_analysis)

print("\n💰 Department-Wise Salary Aggregation:")
print(salary_aggregation)

print("\n📈 Employees Sorted by Projects Completed:")
print(sorted_employees[['Employee_ID', 'Department', 'Projects_Completed']])

print("\n🧮 Average Salary by Department and Performance (Pivot Table):")
print(salary_pivot)



🎯 Employee Performance Classification:
  Employee_ID  Projects_Completed      Performance
0        E001                   3          Average
1        E002                   2  Underperforming
2        E003                   5   High Performer
3        E004                   1  Underperforming
4        E005                   4          Average

📊 Department-Wise Performance Analysis:
  Department  Avg_Monthly_Hours_Worked  Avg_Projects_Completed
0         HR                     140.0                     1.0
1         IT                     180.0                     5.0
2  Marketing                     150.0                     2.0
3      Sales                     165.0                     3.5

💰 Department-Wise Salary Aggregation:
  Department  Total_Salary  Average_Salary  Max_Salary
0         HR          3500          3500.0        3500
1         IT          6000          6000.0        6000
2  Marketing          3800          3800.0        3800
3      Sales          8200          410